> Scrape this Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In [36]:
import pandas as pd # library for data analysis
import numpy as np # library to handle data in a vectorized manner

In [37]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)

In [39]:
df=dfs[0]
df.rename(columns={'Postcode':'PostalCode'}, inplace=True) #The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
print (df.shape) 
df.head()

(288, 3)


,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


> Ignore cells with a borough that is Not assigned.

In [40]:
df = df[df.Borough != "Not assigned"]
print (df.shape) 
df.head()

(211, 3)


,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


> notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
> These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [41]:
df = df.groupby('PostalCode').agg({ 'Borough': 'first', 'Neighbourhood':', '.join }).reset_index()
print (df.shape) 
df.head()

(103, 3)


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


> If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [42]:
df[df.Neighbourhood == "Not assigned"]

,PostalCode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


In [43]:
df.loc[df.Neighbourhood == "Not assigned", 'Neighbourhood'] = df.Borough
df[df.Neighbourhood == "Not assigned"] # check if it is still there

,PostalCode,Borough,Neighbourhood


In [44]:
df[df.PostalCode == "M7A"] #confirm replacement 

,PostalCode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [45]:
df.shape

(103, 3)

In [46]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

In [47]:
#conda install -y -c conda-forge geocoder

In [48]:
import geocoder # import geocoder

In [49]:
# initialize your variable to None
lat_lng_coords = None
postal_code = "M5G"
i=0

# loop until you get the coordinates
while(lat_lng_coords is None):
    i = i + 1
    print(i, '{}, Toronto, Ontario'.format(postal_code))
    if i>5:
        break
    g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    lat_lng_coords = g.latlng

if(lat_lng_coords):
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    print("lat ",latitude," lon ",longitude)

#geocoder does not produce results, will import CSV

1 M5G, Toronto, Ontario
2 M5G, Toronto, Ontario
3 M5G, Toronto, Ontario
4 M5G, Toronto, Ontario
5 M5G, Toronto, Ontario
6 M5G, Toronto, Ontario


> link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [55]:
dfcodes = pd.read_csv('http://cocl.us/Geospatial_data', header=0)
print(dfcodes.shape)
dfcodes.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [56]:
dfmerge = pd.concat([df, dfcodes], axis=1)
dfmerge.head()

,PostalCode,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [57]:
dfmerge=pd.DataFrame(dfmerge,columns=['PostalCode','Borough','Neighbourhood','Latitude','Longitude'])
dfmerge.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
